In [2]:
%%time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader   #  모델을 학습하기 위해서는 지속적으로 데이터를 모델에 전달해야 함
                                          #  또한 데이터를 하나씩 전달하지 않고 원하는 배치 사이즈대로 묶어서 전달하거나
                                          #  더 효율적인 학습을 위해 데이터를 어떤 규칙에 따라 정렬하거나 섞거나 해야함
                                          #  이런 역할을 해주는 것이 DataLoader 모듈

Wall time: 543 ms


In [11]:
batch_size     =   256
learning_rate  =   0.0002
num_epoch      =   10

In [12]:
#  가로, 세로 길이 28의 숫자 이미지로 이루어진 데이터셋
#  각 숫자는 0 ~ 9 까지 손으로 쓴 이미지

mnist_train = dset.MNIST("./", train = True, transform = transforms.ToTensor(),
                        target_transform = None, download = True)
mnist_test = dset.MNIST("./", train = False, transform = transforms.ToTensor(),
                       target_transform = None, download = True)

# 셔플 여부 (shuffle), 데이터를 묶을 때 사용할 프로세스 개수 (num_workers), 
# 묶고 남는 데이터는 버릴지 여부 (drip_last) 등에 대한 설정

train_loader = torch.utils.data.DataLoader(mnist_train, batch_size = batch_size,
                                          shuffle = True, num_workers = 2, drop_last = True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size = batch_size,
                                         shuffle = False, num_workers = 2, drop_last = True)

In [9]:
train_loader

In [6]:
mnist_test

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./
    Split: Test
    StandardTransform
Transform: ToTensor()

## CNN

In [1]:
class CNN(nn.Module):
    def __init__(self):
        #  CNN 클래스의 부모 클래스인 nn.Module을 초기화하는 역할
        super(CNN, self).__init__() 
        
        '''
        * nn.Conv2d()
          in_channels   :   input 이미지의 채널 수
                            => RGB라면 3
          out_channels  :   convolution에 의해서 생성된 channel 수
          
          => Input  : [batch_size, in_channels, 가로, 세로]
          => Output : [batch_size, out_channels, 가로, 세로] 
          
          kernel_size   :  convolution_kenel의 크기. 보통의 filter라고 부름 
          stride        :  convolution의 stride
          padding       :  zero padding을 input을 양쪽에 인자 만큼 해준다
        '''
        
        '''
        * 이미지 크기 계산법 (Output)
          O = floor( ( (1 - K + 2P) / S ) + 1 )
          ★★★★★공식
        
          한 변의 크기 I == 28
          필터의 크기 K == 5
          패딩은 없으므로 P == 0
          stride는 따로 설정 안 했으므로 default 값인 1
          
          ==> O == 24 라는 결과값 (이미지의 크기) 가 나오게 된다.
        '''
        
        '''
        * MaxPool2d()
          kernel_size  :  풀링 연산을 할 때 한 번에 훑는 영역의 크기
        '''
        
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,5),
            nn.ReLU(),
            nn.Conv2d(16,32,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            # [batch_size, 32, 20, 20] => [batch_size, 32, 10, 10]
            
            nn.Conv2d(32,64,5),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
            
            # [batch_size, 32, 10, 10] => [batch_size, 32, 6, 6] => [batch_size, 64, 3, 3]
        )
        
        # Linear 함수를 통해 뉴런의 수를 줄여나간다.
        self.fc_layer = nn.Sequential(
            nn.Linear(64*3*3, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )
        # Output : [batch_size, 10] 형태의 텐서
        
    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1)  # 여기서 -1은 -1인 부분은 알아서 계산하라는 의미 (reshape()와 가틈)
        out = self.fc_layer(out)
        return out 

NameError: name 'nn' is not defined

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # SGD가 아닌 Adam 알고리즘 사용

In [18]:
%%time
loss_arr = []
for i in range(num_epoch):
    for j, [image,label] in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
        if j % 1000 == 0:
            print(loss)                                                
            loss_arr.append(loss.cpu().detach().numpy())

tensor(2.3026, grad_fn=<NllLossBackward>)
tensor(0.2125, grad_fn=<NllLossBackward>)
tensor(0.1283, grad_fn=<NllLossBackward>)
tensor(0.1081, grad_fn=<NllLossBackward>)
tensor(0.0557, grad_fn=<NllLossBackward>)
tensor(0.0558, grad_fn=<NllLossBackward>)
tensor(0.0421, grad_fn=<NllLossBackward>)
tensor(0.0320, grad_fn=<NllLossBackward>)
tensor(0.0390, grad_fn=<NllLossBackward>)
tensor(0.0297, grad_fn=<NllLossBackward>)


In [24]:
correct = 0
total = 0

with torch.no_grad():
    for image, label in test_loader:
        x = image.to(device)
        y_ = label.to(device)
        
        output = model.forward(x)
        _,output_index = torch.max(output, 1)
        
        total += label.size(0)
        correct += (output_index == y_).sum().float()
        
    print("Auucracy of Test Data: {}".format(100 * correct / total))

Auucracy of Test Data: 98.89823913574219


In [21]:
%%time
for j, [image,label] in enumerate(train_loader):
    print(image[0].dtype)
    print(image[0][0].dtype)
    print(image[0][0][1].dtype)
    print(len(image[0]))
    print(len(image[0][0]))
    print(len(image[0][0][1]))
    print(image.dtype)
    print(image.shape)
    print(label)
    print(len(label))
    break

torch.float32
torch.float32
torch.float32
1
28
28
torch.float32
torch.Size([256, 1, 28, 28])
tensor([1, 9, 9, 9, 0, 4, 5, 0, 1, 4, 1, 5, 8, 4, 0, 4, 0, 4, 5, 7, 2, 9, 9, 9,
        7, 7, 1, 8, 3, 6, 0, 5, 7, 2, 8, 1, 9, 0, 9, 3, 3, 6, 6, 9, 0, 1, 4, 2,
        2, 3, 6, 8, 2, 0, 9, 7, 1, 9, 1, 5, 8, 3, 2, 6, 2, 4, 1, 6, 8, 8, 3, 3,
        1, 2, 7, 9, 1, 3, 1, 6, 7, 6, 7, 6, 5, 6, 7, 4, 2, 1, 5, 7, 2, 8, 5, 9,
        1, 9, 6, 6, 9, 3, 0, 6, 8, 3, 3, 6, 9, 1, 4, 4, 2, 7, 6, 9, 3, 6, 6, 5,
        1, 0, 3, 0, 4, 3, 8, 8, 3, 3, 4, 2, 9, 3, 8, 1, 3, 8, 3, 4, 0, 6, 4, 9,
        5, 1, 7, 4, 8, 2, 3, 9, 7, 3, 2, 1, 6, 8, 4, 4, 2, 5, 8, 2, 8, 2, 7, 8,
        9, 2, 7, 3, 2, 7, 7, 7, 5, 0, 8, 6, 5, 5, 1, 9, 8, 6, 6, 3, 2, 4, 9, 4,
        1, 1, 0, 9, 8, 6, 1, 7, 5, 3, 5, 5, 9, 7, 0, 1, 1, 7, 6, 2, 1, 2, 2, 5,
        3, 1, 6, 9, 0, 9, 7, 3, 1, 9, 5, 2, 8, 1, 1, 3, 9, 5, 1, 7, 8, 3, 9, 7,
        6, 1, 4, 4, 9, 0, 2, 9, 9, 5, 0, 6, 3, 3, 0, 4])
256
Wall time: 931 ms


28 x 28 사이즈인데, 256개를 하나로 묶고 정답 레이블을 label로 만들어 뒀구나!!